In [1]:
import sys
import sqlite3
from PyQt5.QtWidgets import QApplication, QMainWindow
from PyQt5.QtGui import QStandardItemModel
from PyQt5.uic import loadUi
from PyQt5.QtCore import pyqtSlot, Qt
from dbhandler import DBHandler
from cetizen_crawler import PnoCrawler, ReleasePriceCrawler, UsedPriceCrawler
from cetizen_model import CashFlowRVI_KT
from datetime import datetime
import pandas as pd

__author__ = 'Sangjin Lee <lee3jjang@gmail.com>'

In [2]:
class CetizenWindow(QMainWindow):

    def __init__(self, parent=None):
        super().__init__(parent)
        conn = sqlite3.connect('cetizen.db')
        self.db = DBHandler(conn)
        self.ui = loadUi('cetizen.ui', self)
        self.model = CashFlowRVI_KT(self.db)
        self.lastModifiedBy = 'USER'
        self.lastUpdateDate = datetime.now().strftime('%Y-%m-%d')

        # 기초데이터
        self.logger1.setText('')
        self.table1.setAlternatingRowColors(True)
        self.table1.setRootIsDecorated(False)
        
        # 가정설정
        self.logger2.setText('')
        self.table2.setAlternatingRowColors(True)
        self.table2.setRootIsDecorated(False)
        
        # 가치평가
        self.logger3.setText('')
        self.table3.setAlternatingRowColors(True)
        self.table3.setRootIsDecorated(False)

        self.setWindowTitle('Cetizen Crawler')
        self.ui.show()

    @pyqtSlot()
    def slot_pno_select(self):
        self.select('상품정보', self.table1)
        self.logger1.append('-- 상품정보 조회 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_pno_insert(self):
        self.logger1.append('-- 상품정보    수집 시작 -- {}'.format(datetime.now()))
        pno_crawler = PnoCrawler()
        df = pno_crawler.crawling(save=True)
        self.logger1.append('-- 상품정보    수집 완료 -- {}'.format(datetime.now()))
        self.db.insTable('상품정보', df)
        self.logger1.append('-- 상품정보    적재 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_pno_delete(self):
        self.db.delTableAll('상품정보')
        self.logger1.append('-- 상품정보    삭제 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_rel_select(self):
        self.select('출고가정보', self.table1)
        self.logger1.append('-- 출고가정보 조회 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_rel_insert(self):
        self.logger1.append('-- 출고가정보 수집 시작 -- {}'.format(datetime.now()))
        pno = list(self.db.getTableAll('상품정보')['PNO'])[:10]
        release_price_crawler = ReleasePriceCrawler(pno)
        df = release_price_crawler.crawling(save=True)
        self.logger1.append('-- 출고가정보 수집 완료 -- {}'.format(datetime.now()))
        self.db.insTable('출고가정보', df)
        self.logger1.append('-- 출고가정보 적재 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_rel_delete(self):
        self.db.delTableAll('출고가정보')
        self.logger1.append('-- 출고가정보 삭제 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_used_select(self):
        self.select('중고가정보', self.table1)
        self.logger1.append('-- 중고가정보 조회 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_used_insert(self):
        self.logger1.append('-- 중고가정보 수집 시작 -- {}'.format(datetime.now()))
        pno = list(self.db.getTableAll('상품정보')['PNO'])[:10]
        used_price_crawler = UsedPriceCrawler(pno)
        df = used_price_crawler.crawling(save=True)
        self.logger1.append('-- 중고가정보 수집 완료 -- {}'.format(datetime.now()))
        self.db.insTable('중고가정보', df)
        self.logger1.append('-- 중고가정보 적재 완료 -- {}'.format(datetime.now()))

    @pyqtSlot()
    def slot_used_delete(self):
        self.db.delTableAll('중고가정보')
        self.logger1.append('-- 중고가정보 삭제 완료 -- {}'.format(datetime.now()))
           
    @pyqtSlot()
    def slot_cont_select(self):
        self.select('계약정보', self.table2)
        self.logger2.append('-- 계약정보 조회 -- {}'.format(datetime.now()))
    
    @pyqtSlot()
    def slot_cont_insert(self):
        df = pd.DataFrame({
            '계약정보ID': [self.cont_id.text()],
            '월납보험료': [float(self.prem.text())],
            '출고가': [float(self.rel_price.text())],
            '반납시점': [float(self.maturity.text())],
            '보장율': [float(self.covered.text())],
            '손실한도': [float(self.loss_cap.text())],
            '공임비': [float(self.op_expen.text())],
            '공동인수율': [float(self.coins.text())],
            '대리점수수료율': [float(self.comm.text())],
            '출재율': [float(self.ceded.text())],
            '출재수수료율': [float(self.reins_comm.text())],
            'LAST_MODIFIED_BY': [self.lastModifiedBy],
            'LAST_UPDATE_DATE': [self.lastUpdateDate]
        })
        self.db.insTable('계약정보', df)
        self.logger2.append('-- 계약정보 삽입 -- {}'.format(datetime.now()))
    
    @pyqtSlot()
    def slot_cont_delete(self):
        pass
    
    @pyqtSlot()
    def slot_assum_select(self):
        self.select('가정', self.table2)
        self.logger2.append('-- 가정 조회 -- {}'.format(datetime.now()))
        
    @pyqtSlot()
    def slot_assum_insert(self):
        df = pd.DataFrame({
            '가정ID': [self.assum_id.text()],
            '총가입자수': [float(self.insured.text())],
            '월해지율': [float(self.lapse.text())],
            '매각가율': [float(self.disc.text())],
            '유지자기변율': [float(self.exer.text())],
            '기타사업비율': [float(self.expen.text())],
            '잔존율': [float(self.resid.text())],
            'LAST_MODIFIED_BY': [self.lastModifiedBy],
            'LAST_UPDATE_DATE': [self.lastUpdateDate]
        })
        self.db.insTable('가정', df)
        self.logger2.append('-- 가정 삽입 -- {}'.format(datetime.now()))
    
    @pyqtSlot()
    def slot_assum_delete(self):
        pass
    
    @pyqtSlot()
    def slot_sensitivity(self):
        self.logger3.append('-- 민감도분석 시작 -- {}'.format(datetime.now()))
        self.model.set_info(self.cont_id_input.text(), self.assum_id_input.text())
        가정_민감도 = self.model.genSce(
            float(self.insured_lb.text()), float(self.insured_ub.text()), float(self.insured_unit.text()), self.insured_checked.isChecked(),
            float(self.lapse_lb.text()), float(self.lapse_ub.text()), float(self.lapse_unit.text()), self.lapse_checked.isChecked(), 
            float(self.disc_lb.text()), float(self.disc_ub.text()), float(self.disc_unit.text()), self.disc_checked.isChecked(), 
            float(self.exer_lb.text()), float(self.exer_ub.text()), float(self.exer_unit.text()), self.exer_checked.isChecked(), 
            float(self.expen_lb.text()), float(self.expen_ub.text()), float(self.expen_unit.text()), self.expen_checked.isChecked(), 
            float(self.resid_lb.text()), float(self.resid_ub.text()), float(self.resid_unit.text()), self.resid_checked.isChecked()
        )
        민감도분석 = self.model.stress_test(가정_민감도)
        self.db.insTable('가정', 가정_민감도)
        self.db.insTable('손익분석', 민감도분석)
        self.logger3.append('-- 민감도분석 완료 -- {}'.format(datetime.now()))
        
    @pyqtSlot()
    def slot_valuation(self):
        self.logger3.append('-- 가치평가 시작 -- {}'.format(datetime.now()))
        self.model.set_info(self.cont_id_input.text(), self.assum_id_input.text())
        가치평가 = self.model.valuation()
        손익분석 = self.model.profit()
        self.db.insTable('가치평가', 가치평가)
        self.db.insTable('손익분석', 손익분석)
        self.logger3.append('-- 가치평가 완료 -- {}'.format(datetime.now()))
        
    @pyqtSlot()
    def slot_valuation_select(self):
        self.select('가치평가', self.table3)
        self.logger3.append('-- 가치평가 조회 -- {}'.format(datetime.now()))
        
    @pyqtSlot()
    def slot_valuation_delete(self):
        self.db.delTableAll('가치평가')
        self.logger3.append('-- 가치평가 삭제 -- {}'.format(datetime.now()))
    
    @pyqtSlot()
    def slot_profit_select(self):
        self.select('손익분석', self.table3)
        self.logger3.append('-- 손익분석 조회 -- {}'.format(datetime.now()))
        
    @pyqtSlot()
    def slot_profit_delete(self):
        self.db.delTableAll('손익분석')
        self.logger3.append('-- 손익분석 삭제 -- {}'.format(datetime.now()))
    
    def select(self, tableName, table):
        df = self.db.getTableAll(tableName)
        col_name = df.columns[:-2]
        m = len(col_name)
        n = df.shape[0]
        rows = QStandardItemModel(n, m, self)
        for j in range(m):
            rows.setHeaderData(j, Qt.Horizontal, col_name[j])
        for i in range(n):
            for j in range(m):
                rows.setData(rows.index(i, j), str(df.iloc[i, j]))
        table.setModel(rows)

In [3]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    cetizen = CetizenWindow()
    
    #가정ID = 'TEST_ASSUM_001'
    #계약정보ID = 'TEST_CONT_001'
    #conn = sqlite3.connect('cetizen.db')
    #db = DBHandler(conn)
    #model = CashFlowRVI_KT(db)
    #model.set_info(계약정보ID, 가정ID)
    
    sys.exit(app.exec_())

IntegrityError: UNIQUE constraint failed: 가정.가정ID

SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
